In [ ]:
!pip install pytorch-lightning albumentations tensorboard -q

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import albumentations as A
from albumentations.pytorch import ToTensorV2

import os
import json
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

print(f"PyTorch version: {torch.__version__}")
print(f"PyTorch Lightning version: {pl.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

PyTorch version: 2.8.0+cu126
PyTorch Lightning version: 2.5.5
CUDA available: True


In [ ]:
DATA_ROOT = '/content/Dataset_Robomaster-1'  # Modify this path
CHECKPOINT_DIR = '/content/checkpoints'
LOG_DIR = '/content/logs'

In [ ]:
# Small U-Net Architecture

class DoubleConv(nn.Module):
    """Double convolution block with optional dropout"""
    def __init__(self, in_channels, out_channels, dropout=0.0):
        super().__init__()
        layers = [
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        ]

        if dropout > 0:
            layers.append(nn.Dropout2d(dropout))

        layers.extend([
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        ])

        self.double_conv = nn.Sequential(*layers)

    def forward(self, x):
        return self.double_conv(x)


class SmallUNet(nn.Module):
    """Lightweight U-Net for small datasets"""
    def __init__(self, in_channels=3, out_channels=1, base_channels=32):
        super().__init__()

        # Encoder
        self.enc1 = DoubleConv(in_channels, base_channels, dropout=0.1)
        self.pool1 = nn.MaxPool2d(2)
        self.enc2 = DoubleConv(base_channels, base_channels*2, dropout=0.1)
        self.pool2 = nn.MaxPool2d(2)
        self.enc3 = DoubleConv(base_channels*2, base_channels*4, dropout=0.2)
        self.pool3 = nn.MaxPool2d(2)

        # Bottleneck
        self.bottleneck = DoubleConv(base_channels*4, base_channels*8, dropout=0.2)

        # Decoder
        self.upconv3 = nn.ConvTranspose2d(base_channels*8, base_channels*4, kernel_size=2, stride=2)
        self.dec3 = DoubleConv(base_channels*8, base_channels*4, dropout=0.2)
        self.upconv2 = nn.ConvTranspose2d(base_channels*4, base_channels*2, kernel_size=2, stride=2)
        self.dec2 = DoubleConv(base_channels*4, base_channels*2, dropout=0.1)
        self.upconv1 = nn.ConvTranspose2d(base_channels*2, base_channels, kernel_size=2, stride=2)
        self.dec1 = DoubleConv(base_channels*2, base_channels, dropout=0.1)

        # Output
        self.out = nn.Conv2d(base_channels, out_channels, kernel_size=1)

    def forward(self, x):
        # Encoder
        enc1 = self.enc1(x)
        enc2 = self.enc2(self.pool1(enc1))
        enc3 = self.enc3(self.pool2(enc2))

        # Bottleneck
        bottleneck = self.bottleneck(self.pool3(enc3))

        # Decoder
        dec3 = self.upconv3(bottleneck)
        dec3 = torch.cat([dec3, enc3], dim=1)
        dec3 = self.dec3(dec3)

        dec2 = self.upconv2(dec3)
        dec2 = torch.cat([dec2, enc2], dim=1)
        dec2 = self.dec2(dec2)

        dec1 = self.upconv1(dec2)
        dec1 = torch.cat([dec1, enc1], dim=1)
        dec1 = self.dec1(dec1)

        return self.out(dec1)

In [ ]:
# Dataloader

class ArmorPlateDataset(Dataset):
    """Dataset for armor plate segmentation from COCO format"""

    def __init__(self, root_dir, split='train', augment=False):
        self.root_dir = root_dir
        self.split = split
        self.img_dir = os.path.join(root_dir, split)

        # Load COCO annotations
        ann_path = os.path.join(self.img_dir, '_annotations.coco.json')
        with open(ann_path, 'r') as f:
            self.coco_data = json.load(f)

        # Create image_id to annotations mapping
        self.img_id_to_anns = {}
        for ann in self.coco_data['annotations']:
            img_id = ann['image_id']
            if img_id not in self.img_id_to_anns:
                self.img_id_to_anns[img_id] = []
            self.img_id_to_anns[img_id].append(ann)

        self.images = self.coco_data['images']

        # Albumentations transforms
        if augment:
            self.transform = A.Compose([
                A.HorizontalFlip(p=0.5),
                A.VerticalFlip(p=0.5),
                A.Rotate(limit=15, p=0.5),
                A.RandomBrightnessContrast(p=0.3),
                A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                ToTensorV2(),
            ])
        else:
            self.transform = A.Compose([
                A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                ToTensorV2(),
            ])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_info = self.images[idx]
        img_id = img_info['id']
        img_filename = img_info['file_name']
        img_height = img_info['height']
        img_width = img_info['width']

        # Load image
        img_path = os.path.join(self.img_dir, img_filename)
        image = np.array(Image.open(img_path).convert('RGB'))

        # Create binary mask from bounding boxes
        mask = np.zeros((img_height, img_width), dtype=np.float32)
        if img_id in self.img_id_to_anns:
            for ann in self.img_id_to_anns[img_id]:
                bbox = ann['bbox']
                x, y, w, h = map(int, bbox)
                mask[y:y+h, x:x+w] = 1.0

        # Apply transforms
        transformed = self.transform(image=image, mask=mask)
        image = transformed['image']
        mask = transformed['mask'].unsqueeze(0)

        return image, mask

In [ ]:
# Lighting Datamodule

class ArmorDataModule(pl.LightningDataModule):
    """Lightning DataModule for armor detection"""

    def __init__(self, data_root, batch_size=8, num_workers=2):
        super().__init__()
        self.data_root = data_root
        self.batch_size = batch_size
        self.num_workers = num_workers

    def setup(self, stage=None):
        if stage == 'fit' or stage is None:
            self.train_dataset = ArmorPlateDataset(self.data_root, split='train', augment=True)
            self.val_dataset = ArmorPlateDataset(self.data_root, split='valid', augment=False)

        if stage == 'test' or stage is None:
            self.test_dataset = ArmorPlateDataset(self.data_root, split='test', augment=False)

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers,
            pin_memory=True
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            pin_memory=True
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            pin_memory=True
        )


In [ ]:
# Training, Validation, and Test via Pytorch Lightning

def dice_coefficient(pred, target, smooth=1e-6):
    """Calculate Dice coefficient"""
    pred = torch.sigmoid(pred)
    pred = (pred > 0.5).float()

    intersection = (pred * target).sum(dim=(2, 3))
    union = pred.sum(dim=(2, 3)) + target.sum(dim=(2, 3))

    dice = (2. * intersection + smooth) / (union + smooth)
    return dice.mean()


class ArmorUNet(pl.LightningModule):
    """PyTorch Lightning module for armor plate detection"""

    def __init__(self, learning_rate=1e-4, weight_decay=1e-5, base_channels=32):
        super().__init__()
        self.save_hyperparameters()

        # Model
        self.model = SmallUNet(in_channels=3, out_channels=1, base_channels=base_channels)

        # Loss
        self.criterion = nn.BCEWithLogitsLoss()

        # For visualization
        self.example_input_array = torch.randn(1, 3, 640, 640)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, masks = batch
        outputs = self(images)

        loss = self.criterion(outputs, masks)
        dice = dice_coefficient(outputs, masks)

        # Log metrics
        self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log('train_dice', dice, on_step=False, on_epoch=True, prog_bar=True)

        return loss

    def validation_step(self, batch, batch_idx):
        images, masks = batch
        outputs = self(images)

        loss = self.criterion(outputs, masks)
        dice = dice_coefficient(outputs, masks)

        # Log metrics
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log('val_dice', dice, on_step=False, on_epoch=True, prog_bar=True)

        return {'val_loss': loss, 'val_dice': dice}

    def test_step(self, batch, batch_idx):
        images, masks = batch
        outputs = self(images)

        loss = self.criterion(outputs, masks)
        dice = dice_coefficient(outputs, masks)

        self.log('test_loss', loss)
        self.log('test_dice', dice)

        return {'test_loss': loss, 'test_dice': dice}

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            self.parameters(),
            lr=self.hparams.learning_rate,
            weight_decay=self.hparams.weight_decay
        )

        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode='max',
            factor=0.5,
            patience=10,
            # verbose=True  # Removed verbose as it's not supported in recent PyTorch versions
        )

        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': scheduler,
                'monitor': 'val_dice',
                'interval': 'epoch',
                'frequency': 1
            }
        }

# Optional Visualization Tool

def plot_training_history(checkpoint_path):
    """Plot training history from checkpoint"""
    checkpoint = torch.load(checkpoint_path)

    # Lightning saves metrics in a different format
    # You'll need to use TensorBoard or load from logger
    print("Use TensorBoard to visualize training:")
    print(f"  tensorboard --logdir {LOG_DIR}")


def visualize_predictions(model, datamodule, num_samples=4):
    """Visualize model predictions"""
    model.eval()
    datamodule.setup('test')
    test_loader = datamodule.test_dataloader()

    images, masks = next(iter(test_loader))
    images = images[:num_samples]
    masks = masks[:num_samples]

    with torch.no_grad():
        outputs = model(images)
        preds = torch.sigmoid(outputs) > 0.5

    fig, axes = plt.subplots(num_samples, 3, figsize=(12, 4*num_samples))

    for i in range(num_samples):
        # Denormalize
        img = images[i].cpu().numpy().transpose(1, 2, 0)
        img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
        img = np.clip(img, 0, 1)

        # Plot
        axes[i, 0].imshow(img)
        axes[i, 0].set_title('Input Image')
        axes[i, 0].axis('off')

        axes[i, 1].imshow(masks[i].cpu().squeeze(), cmap='gray')
        axes[i, 1].set_title('Ground Truth')
        axes[i, 1].axis('off')

        axes[i, 2].imshow(preds[i].cpu().squeeze(), cmap='gray')
        axes[i, 2].set_title('Prediction')
        axes[i, 2].axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
# Training Loop Defined

def train_armor_detector(
    data_root=DATA_ROOT,
    batch_size=8,
    max_epochs=75,
    learning_rate=1e-4,
    base_channels=32,
    checkpoint_dir=CHECKPOINT_DIR,
    log_dir=LOG_DIR
):
    """Complete training pipeline with PyTorch Lightning"""

    # Set seeds
    pl.seed_everything(42, workers=True)

    print("="*80)
    print("ARMOR PLATE DETECTION - PyTorch Lightning")
    print("="*80)

    # Data module
    print("\nInitializing data module...")
    datamodule = ArmorDataModule(
        data_root=data_root,
        batch_size=batch_size,
        num_workers=2
    )

    # Model
    print("Creating model...")
    model = ArmorUNet(
        learning_rate=learning_rate,
        base_channels=base_channels
    )

    print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

    # Callbacks
    checkpoint_callback = ModelCheckpoint(
        dirpath=checkpoint_dir,
        filename='armor-unet-{epoch:02d}-{val_dice:.4f}',
        monitor='val_dice',
        mode='max',
        save_top_k=1,
        verbose=True
    )

    early_stop_callback = EarlyStopping(
        monitor='val_dice',
        patience=15,
        mode='max',
        verbose=True
    )

    lr_monitor = LearningRateMonitor(logging_interval='epoch')

    # Logger
    logger = TensorBoardLogger(log_dir, name='armor_unet')

    # Trainer
    trainer = pl.Trainer(
        max_epochs=max_epochs,
        accelerator='gpu' if torch.cuda.is_available() else 'cpu',
        devices=1,
        callbacks=[checkpoint_callback, early_stop_callback, lr_monitor],
        logger=logger,
        log_every_n_steps=10,
        deterministic=True
    )

    # Train
    print("\nStarting training...")
    print("="*80)
    trainer.fit(model, datamodule)

    # Test
    print("\n" + "="*80)
    print("Testing best model...")
    trainer.test(model, datamodule, ckpt_path='best')

    print("\n" + "="*80)
    print("Training complete!")
    print(f"Best model saved to: {checkpoint_callback.best_model_path}")
    print(f"Best validation Dice: {checkpoint_callback.best_model_score:.4f}")
    print(f"\nView training in TensorBoard:")
    print(f"  tensorboard --logdir {log_dir}")

    return model, trainer, datamodule

In [ ]:
# Training

model, trainer, datamodule = train_armor_detector(
    data_root=DATA_ROOT,
    batch_size=8,
    max_epochs=75,
    learning_rate=1e-4,
    base_channels=32
)

# Visualize predictions
print("\nVisualizing predictions...")
visualize_predictions(model, datamodule, num_samples=4)


INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


ARMOR PLATE DETECTION - PyTorch Lightning

Initializing data module...
Creating model...
Model parameters: 1,928,417

Starting training...


INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type              | Params | Mode  | In sizes         | Out sizes       
----------------------------------------------------------------------------------------------
0 | model     | SmallUNet         | 1.9 M  | train | [1, 3, 640, 640] | [1, 1, 640, 640]
1 | criterion | BCEWithLogitsLoss | 0      | train | ?                | ?               
----------------------------------------------------------------------------------------------
1.9 M     Trainable params
0         Non-trainable params
1.9 M     Total params
7.714     Total estimated model params size (MB)
72        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 57, global step 2552: 'val_dice' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 58, global step 2596: 'val_dice' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.callbacks.early_stopping:Metric val_dice improved by 0.001 >= min_delta = 0.0. New best score: 0.887
INFO:pytorch_lightning.utilities.rank_zero:Epoch 59, global step 2640: 'val_dice' reached 0.88688 (best 0.88688), saving model to '/content/checkpoints/armor-unet-epoch=59-val_dice=0.8869.ckpt' as top 1
